In [ ]:
library(tidyverse)
library(lubridate)
library(janitor)
library(skimr)
library(psych)
library(ggplot2)
library(reshape2)
library(DataExplorer)
library(scales)

In [ ]:
stores <- read.csv("stores-data-set.csv")
features <- read.csv("Features-data-set.csv")
sales <- read.csv("sales-data-set.csv")

Descriptions of the datasets.

In [ ]:
head(stores)
head(features)
head(sales)

In [ ]:
glimpse(stores)
glimpse(features)
glimpse(sales)

Confirm the distinct stores and departments.

In [ ]:
n_distinct(stores$Store) #checking store Ids 

n_distinct(features$Date) #checking before merging with sales

n_distinct(sales$Date)
n_distinct(sales$Store)
n_distinct(sales$Dept)

In [ ]:
sum(is.na(sales))
sum(is.na(features))
sum(is.na(sales))

In [ ]:
features$Date <- as.Date(features$Date, "%d/%m/%Y")
sales$Date <- as.Date(sales$Date, "%d/%m/%Y")

merge features frames  and sales frames

In [ ]:
features_date <- aggregate(list("Temperature" = features$Temperature, "Fuel_Price" = features$Fuel_Price, "CPI" = features$CPI, "Unemployment" = features$Unemployment), by = list("Date" = features$Date), FUN=mean, na.rm = TRUE)

features_date <- features_date %>% 
  mutate(aggregate(list("IsHoliday" = features$IsHoliday), by = list("Date" = features$Date), FUN = sum, na.rm = TRUE))

In [ ]:
# Data is already sorted by Dates
sales_date <- aggregate(list("Weekly_Sales" = sales$Weekly_Sales), by= list("Date" = sales$Date), FUN=sum, na.rm = TRUE)
# Converting sales into Millions
sales_date$Weekly_Sales <- as.integer(sales_date$Weekly_Sales / 1000000)

In [ ]:
# Merging the features_date and sales_date datasets
sales_complete <- merge(sales_date, features_date, by = "Date", all.sales_date = TRUE)
head(sales_complete)

# 5. Data analysis phase in terms of correlations.

In [ ]:
# build a density plot
sales_complete %>% plot_density(ncol = 4, ggtheme = theme_minimal())
# build correlation plot
corPlot(sales_complete[,2:7], upper = FALSE, scale = FALSE, main = "Correlation in Sales Attributes")

Plot and sales graphs

In [ ]:
plot_sales <- gather(sales_complete, "attribs", "Value", -Date, -IsHoliday) 

ggplot(plot_sales, aes(Date, Value)) + geom_line(aes(color = Value), linewidth = 1) + facet_grid(attribs~., scales = "free_y", #adjust scales
                                                                                  switch = "y") + #switch y-axis labels to left
  ylab(NULL) + #remove the label "Value"
  theme(strip.background = element_blank(), #remove the background
        strip.placement = "outside", strip.text.y.left = element_text(angle = 0), legend.position = "none") + scale_x_date(date_breaks = "5 months", date_labels = '%Y-%m')

In [ ]:
sales_monthly <- sales_complete %>% group_by(month = lubridate::floor_date(Date, "month")) %>% summarize("Weekly_Sales" = sum(Weekly_Sales))

sales_monthly <- sales_monthly %>% mutate(Month = as.integer(lubridate::month(month)), Year = lubridate::year(month)) %>% group_by(Month) %>%
  summarize("Sales" = sum(Weekly_Sales))

In [ ]:
ggplot(sales_monthly, aes(x = Month, y = Sales, fill=as.factor(Sales))) + geom_col() + guides(fill="none") + scale_x_continuous(breaks = c(1,2,3,4,5,6,7,8,9,10,11,12)) + scale_fill_discrete() + labs(title = "Monthly Sales")+ theme(plot.title = element_text(hjust = 0.5))

In [ ]:
sales_yearly <- sales_complete %>% group_by(year = lubridate::floor_date(Date, "year")) %>% summarize("Sales" = sum(Weekly_Sales))

ggplot(sales_yearly, aes(x = year, y = Sales, fill=as.factor(Sales))) + geom_col() + guides(fill="none") + scale_fill_discrete() + labs(title = "Yearly Sales")+ theme(plot.title = element_text(hjust = 0.5))

In [ ]:
stores_agg <- aggregate(list("Temperature" = features$Temperature, "Fuel_Price" = features$Fuel_Price), by = list("Store" = features$Store), FUN=mean, na.rm = TRUE)

stores_agg <- stores_agg %>% mutate(aggregate(list("IsHoliday" = features$IsHoliday), by = list("Store" = features$Store), FUN=sum, na.rm = TRUE))

temp_store <- aggregate(list("Weekly_Sales" = sales$Weekly_Sales), by = list("Store" = sales$Store), FUN=sum, na.rm = TRUE)

temp_store$Weekly_Sales <- as.integer(temp_store$Weekly_Sales / 1000000)

stores <- stores %>% mutate("Temp" = stores_agg$Temperature) 
stores <- stores %>% mutate("Fuel_Price" = stores_agg$Fuel_Price) 
stores <- stores %>% mutate("Holiday" = stores_agg$IsHoliday) 
stores <- stores %>% mutate("Weekly_Sales" = temp_store$Weekly_Sales)

str(stores)

ggplot(stores, aes(x = Type, y = Size, fill = Type)) +  geom_col() + guides(fill="none") + scale_y_continuous(labels=comma)
ggplot(stores, aes(x = Type, y = Weekly_Sales, fill = Type)) + geom_boxplot() + guides(fill="none")

In [ ]:
# Checking number of unique departments
n_distinct(sales$Dept)
#Grouping departments by sales
dept_data <- aggregate(list("Weekly_Sales" = sales$Weekly_Sales), by = list("Dept" = sales$Dept), FUN=sum, na.rm = TRUE)

dept_data$Weekly_Sales <- as.integer(dept_data$Weekly_Sales / 1000000)

dept_data[order(dept_data$Weekly_Sales),]

ggplot(dept_data, aes(x = Dept, y = Weekly_Sales, fill = "light pink")) + geom_col() + labs(title = "Department vs Weekly Sales") + theme(plot.title = element_text(hjust = 0.5)) + guides(fill="none")

dept_data[which.max(dept_data$Weekly_Sales),]
dept_data[which.min(dept_data$Weekly_Sales),]
dept_data[dept_data$Weekly_Sales == 0,]

In [ ]:
# Aggregate weekly sales by Date and Store
sales_date_store <- aggregate(list("Weekly_Sales" = sales$Weekly_Sales), by = list("Date" = sales$Date, "Store" = sales$Store), FUN=sum, na.rm = TRUE)
# Converting weekly sales into millions
sales_date_store$Weekly_Sales <- as.integer(sales_date_store$Weekly_Sales / 1000000)
# Merging features data set with sales_date_store
sales_stores_combined <- merge(features, sales_date_store, by = c("Date", "Store"), all.features = TRUE)

In [ ]:
# Adding Store number and type column in the sales_stores_combined dataset 
sales_stores_combined <- merge(sales_stores_combined, stores[c("Store", "Type")], by = "Store", all.sales_stores_combined = TRUE)

sales_stores_combined <- sales_stores_combined[,-15:-19]
# Omitting all NA values
clean_sales_stores <- na.omit(sales_stores_combined)
# Aggregating average markdown values by Date
markdowns <- aggregate(list("Markdown1" = clean_sales_stores$MarkDown1, "Markdown2" = clean_sales_stores$MarkDown2, "Markdown3" = clean_sales_stores$MarkDown3, "Markdown4" = clean_sales_stores$MarkDown4), by = list("Date" = clean_sales_stores$Date), FUN=mean, na.rm = TRUE)

In [ ]:
# Plotting the markdowns as trending lines

ggplot(markdowns, aes(x = Date, y = Markdowns)) + geom_line(aes(y = Markdown1, colour = "Markdown1"),  linewidth = 1) + geom_line(aes(y = Markdown2, colour = "Markdown2"),linewidth = 1) + geom_line(aes(y = Markdown3, colour = "Markdown3"),linewidth = 1) + geom_line(aes(y = Markdown4, colour = "Markdown4"),linewidth = 1)